In [1]:
import numpy as np
from numpy import shape, matrix, log, exp, zeros,random,dot,multiply
from mnist import readWithoutBias
from math import sqrt

In [26]:
data_train, label_train = readWithoutBias(dataset="training")
data_test, label_test = readWithoutBias(dataset="testing")
label_train = matrix(label_train)
label_test  = matrix(label_test)
data_train  = matrix(data_train).T
data_test   = matrix(data_test).T

In [27]:
train_mean  = data_train.mean(axis=1)
train_std   = data_train.std(axis=1)
data_train  = np.nan_to_num((data_train - train_mean)/train_std)
data_test   = np.nan_to_num((data_test - train_mean)/train_std)

/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()
/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: RuntimeWarning: divide by zero encountered in divide
/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: RuntimeWarning: invalid value encountered in divide


##Initializations

In [28]:
train = zip(data_train.T,label_train)
mini_batch_size = 10

n_input = shape(data_train)[0] # excluding bias term
n_hidden = 30               # excluding bias term
n_output = 10

theta1 = matrix(random.randn(n_hidden,n_input))/sqrt(n_input)
bias1  = matrix(random.randn(n_hidden,1))
theta2 = matrix(random.randn(n_output,n_hidden))/sqrt(n_hidden)
bias2  = matrix(random.randn(n_output,1))

### Training Code

In [29]:
alpha = 0.1
epochs = 10
count = 0
X_test = data_test
y_test = label_test

In [30]:
count = 0
alpha = 0.1
mini_batch_size = 10
old_accuracy = 0
for i in range(epochs):
    random.shuffle(train)
    mini_batches = [train[k:k+mini_batch_size] for k in xrange(0,len(train),mini_batch_size)]
    for mini_batch in mini_batches:
        d1  = np.zeros_like(theta1)
        d2  = np.zeros_like(theta2)
        db1 = np.zeros_like(bias1)
        db2 = np.zeros_like(bias2)
        for X,y in mini_batch:
            gradTheta1, gradBias1, gradTheta2, gradBias2 = \
                backPropGradient(X.T, y, theta1, theta2, bias1, bias2) #just one example passed
            d1  += gradTheta1
            db1 += gradBias1
            d2  += gradTheta2
            db2 += gradBias2
        d1  /= mini_batch_size
        db1 /= mini_batch_size
        d2  /= mini_batch_size
        db2 /= mini_batch_size
        theta1 = theta1 - alpha*d1
        bias1  = bias1  - alpha*db1
        theta2 = theta2 - alpha*d2
        bias2  = bias2  - alpha*db2
        if count%1000==0:
            print "batch num: ",count
        count+=1
    print accuracy(X_test, y_test, theta1, theta2, bias1, bias2)# get error on complete data
    count = 0

batch num:  0
batch num:  1000
batch num:  2000
batch num:  3000
batch num:  4000
batch num:  5000
0.9338
batch num:  0
batch num:  1000
batch num:  2000
batch num:  3000
batch num:  4000
batch num:  5000
0.9404
batch num:  0
batch num:  1000
batch num:  2000
batch num:  3000
batch num:  4000
batch num:  5000
0.945
batch num:  0
batch num:  1000
batch num:  2000
batch num:  3000
batch num:  4000
batch num:  5000
0.9446
batch num:  0
batch num:  1000


/Users/apoorve/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from IPython.kernel.zmq import kernelapp as app


KeyboardInterrupt: 

### Checking backpropgradient vs numerical gradient

In [18]:
X, y = train[0]
X = X.T
gradTheta1, gradBias1, gradTheta2, gradBias2 = backPropGradient(X,y,theta1,theta2, bias1, bias2)
numGrad1,numGradBias1,numGrad2,numGradBias2  = numericalGradient(X,y,theta1,theta2, bias1, bias2)
print np.sum(np.subtract(numGradBias2,gradBias2))
print np.sum(np.subtract(numGradBias1,gradBias1))
print np.sum(np.subtract(numGrad2,gradTheta2))
print np.sum(np.subtract(numGrad1,gradTheta1))

-1.42438449924e-10
7.08770802466e-12
1.64607838818e-09
-8.87563875562e-09


### Helper Functions

In [4]:
def zscore(z):
    return np.nan_to_num((z - z.mean(axis=1))/z.std(axis=1))

In [5]:
def accuracy(X, y, theta1, theta2, bias1, bias2):
    a1 = X
    z2 = dot(theta1,a1) + bias1
    a2 = sigmoid(z2)
    z3 = dot(theta2,a2) + bias2
    a3 = sigmoid(z3)
    
    pred = np.argmax(a3,axis=0)
    return np.sum(np.equal(pred,y.T))/float(len(y))
    

In [6]:
def backPropGradient(X, y, theta1, theta2, bias1, bias2):
    a1 = X
    z2 = dot(theta1,a1) + bias1
    a2 = sigmoid(z2)
    z3 = dot(theta2,a2) + bias2
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    for i in range(len(y)):
        t[y[i],i] = 1
    
    delta3 = a3-t #shape (10,1)
    delta2 = dot(theta2.T,delta3)#shape (100,10)X(10,1) = (100,1)
    delta2 = multiply(delta2,sigmoidGradient(z2))

    gradTheta2 = dot(delta3,a2.T)
    gradBias2  = delta3
    gradTheta1 = dot(delta2,a1.T)
    gradBias1  = delta2
    return gradTheta1, gradBias1, gradTheta2, gradBias2

In [7]:
def errorFn(X, y, theta1, theta2, bias1, bias2):
    n_examples = shape(X)[1]
    a1 = X
    z2 = dot(theta1,a1) + bias1
    a2 = sigmoid(z2)
    z3 = dot(theta2,a2) + bias2
    a3 = sigmoid(z3)
    
    t = np.zeros_like(a3)
    for i in range(len(y)):
        t[y[i],i] = 1

    error = np.sum(multiply(t,log(a3)) + multiply((1-t),log(1-a3)))
    error = -error/n_examples
    return error

In [8]:
def numericalGradient(X, y, theta1, theta2, bias1, bias2):
    epsilon = 10**-5
    numGrad1     = np.zeros_like(theta1)
    numGradBias1 = np.zeros_like(bias1)
    numGrad2     = np.zeros_like(theta2)
    numGradBias2 = np.zeros_like(bias2)
    
    for i in range(shape(theta1)[0]):
        for j in range(shape(theta1)[1]):
            theta1_pos = np.copy(theta1)
            theta1_neg = np.copy(theta1)
            theta1_pos[i,j] += epsilon
            theta1_neg[i,j] -= epsilon
            numGrad1[i,j]    = (errorFn(X, y, theta1_pos, theta2, bias1, bias2) - \
                                errorFn(X, y, theta1_neg, theta2, bias1, bias2))/2/epsilon
    for i in range(shape(theta2)[0]):
        for j in range(shape(theta2)[1]):
            theta2_pos = np.copy(theta2)
            theta2_neg = np.copy(theta2)
            theta2_pos[i,j] += epsilon
            theta2_neg[i,j] -= epsilon
            numGrad2[i,j]    = (errorFn(X, y, theta1, theta2_pos, bias1, bias2) - \
                                errorFn(X, y, theta1, theta2_neg, bias1, bias2))/2/epsilon
    for i in range(shape(bias1)[0]):
        for j in range(shape(bias1)[1]):
            bias1_pos         = np.copy(bias1)
            bias1_neg         = np.copy(bias1)
            bias1_pos[i,j]   += epsilon
            bias1_neg[i,j]   -= epsilon
            numGradBias1[i,j] = (errorFn(X, y, theta1, theta2, bias1_pos, bias2) - \
                                errorFn(X, y, theta1, theta2, bias1_neg, bias2))/2/epsilon
    for i in range(shape(bias2)[0]):
        for j in range(shape(bias2)[1]):
            bias2_pos         = np.copy(bias2)
            bias2_neg         = np.copy(bias2)
            bias2_pos[i,j]   += epsilon
            bias2_neg[i,j]   -= epsilon
            numGradBias2[i,j] = (errorFn(X, y, theta1, theta2, bias1, bias2_pos) - \
                                errorFn(X, y, theta1, theta2, bias1, bias2_neg))/2/epsilon
    return numGrad1,numGradBias1,numGrad2,numGradBias2

In [9]:
def sigmoid(z):
    return 1/(1+exp(-z))
def sigmoidGradient(z):
    s = sigmoid(z)
    return multiply(s,1-s)

In [33]:
def funnyTanh(z):
    return 1.7159 * np.tanh(2.0/3.0 * z) + .001*z
def funnyTanhGradient(z):
    1.7159 * 2.0 / 3.0 * (1.0 / np.cosh(2.0/3.0 * z))**2 + .001

In [ ]:
sigmoid = lambda z: 1.0/(1.0+np.exp(-z))
sigmoid_prime = lambda z: sigmoid(z)*(1-sigmoid(z))
funny_tanh = lambda z: 1.7159 * np.tanh(2.0/3.0 * z) + .001*z
funny_tanh_prime = lambda z: 1.7159 * 2.0 / 3.0 * (1.0 / np.cosh(2.0/3.0 * z))**2 + .001
relu = lambda z: z * (z > 0)
relu_prime = lambda z: z >= 0
leaky_relu = lambda z: np.maximum(.1*z, z)
leaky_relu_prime = lambda z: 1*(z>=0) + .1*(z<0)